# Flow accumulation D∞
The D-infinity (D∞) flow accumulation method is an enhancement of the D8 method. It addresses some of the limitations of the D8 method by considering multiple flow directions from each cell, rather than just the steepest single descent.

In the D∞ method, flow is distributed from each cell to all its neighboring cells based on the relative slope values in each direction. This means that instead of choosing a single direction as in the D8 method, flow is distributed to multiple directions according to the slope gradients. The D∞ method allows for a more comprehensive representation of flow patterns in complex terrain, including flat areas, ridges, and valleys.

Here's a simplified explanation of how the D∞ flow accumulation method works:

- **Determination of Flow Directions**: For each cell in the DEM, the slope gradients are computed in all directions (8 or more, depending on the grid resolution and user preference). This is typically achieved by calculating the slope or gradient for each cell in all directions.

- **Flow Accumulation**: Flow accumulation is calculated by distributing flow from each cell to its neighboring cells based on the relative slopes. Cells with steeper slopes in a particular direction receive more flow from the upstream cell in that direction.

- **Flow Routing**: Similar to the D8 method, flow routing involves tracing the flow path of water across the DEM based on the accumulated flow values. This information can be used for various hydrological analyses, such as watershed delineation, flood modeling, and erosion prediction.

The D∞ method provides a more accurate representation of flow patterns in areas with complex terrain, including flat areas, ridges, and valleys, compared to the D8 method. However, it may also be computationally more intensive due to the need to consider multiple flow directions for each cell. Despite this, the D∞ method is widely used in hydrology and terrain analysis to improve the accuracy of flow accumulation modeling.

## 1. Import libraries

In [51]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import rasterio
from ipywidgets import widgets, interact, Layout, IntSlider # Interactive widgets for Jupyter notebooks
import os
# Import seaborn
import seaborn as sns
# Apply the default theme
sns.set_theme()
import warnings
warnings.filterwarnings("ignore")

## Flow accumlation - D-infinity
This function computes the flow contribution to neighbouring cells based on the slope from the current cell (x, y). It calculates the slope in eight directions using the DEM and normalizes these slopes to get flow contributions. The function returns an array containing these flow contributions.

In [2]:
def flow_contribution_neighbours(x, y, dem):
    """
    Helper function to get the flow contribution to neighbouring cells based on slope.

    Args:
    - x, y: Coordinates of the cell
    - dem: The digital elevation model (DEM)

    Returns:
    - flow_contributions: Array containing the flow contribution to neighbouring cells
    """

    rows, cols = dem.shape  # Get the dimensions of the DEM (rows, cols)

    # Define the neighbourhood directions (8-connected)
    neighbours = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),           (0, 1),
                  (1, -1),  (1, 0),  (1, 1)]

    # Initialize an array to store slopes in all eight directions
    slopes = np.zeros(8)

    # Calculate slopes in all eight directions
    for j, (dx, dy) in enumerate(neighbours):
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols:  # Check if the neighbouring cell is within bounds
            # Calculate the difference in elevation (height difference)
            dz = min(dem[nx, ny] - dem[x, y], 0)  # Take the minimum to account for negative slopes
            # Calculate the slope in this direction
            slopes[j] = dz / np.sqrt(dx ** 2 + dy ** 2)  # Slope = change in height / distance
        else:
            slopes[j] = np.nan  # If the neighbouring cell is out of bounds, assign NaN to its slope

    # Compute the flow contributions based on the slopes
    flow_contributions = slopes / np.nansum(slopes)  # Normalize slopes to get flow contributions

    return flow_contributions

This function calculates flow accumulation using the D-infinity method. It iterates through each 
non-NaN cell, computes flow contributions to its neighbours, updates flow accumulation accordingly, 
and stores upstream and downstream cell indices along with flow contributions. Finally, it returns 
the flow accumulation matrix, upstream and downstream indices, and flow contributions.

In [67]:
from numba import njit
@njit
def flow_accumulation_Dinf(dem, *num_iterations):
    """
    Calculates the flow accumulation for a digital elevation model (DEM) using the D-infinity method.
    
    Args:
        dem (numpy.ndarray): A 2D array representing the elevation data.

    Returns:
        - fa (numpy.ndarray): The flow accumulation values for each cell.
        - ix (list): The upstream cell index for each cell (not including source cells).
        - ixc (list): The downstream cell index for each cell (not including sink cells).
        - fc (list): The flow contributions for each cell.
    """

    # Create a copy of DEM to work with
    dem_temp = dem.copy()
    
    # Initialize the flow accumulation matrix with ones
    fa = np.ones_like(dem, dtype=float)
    fa[np.isnan(dem)] = np.nan
    
    if num_iterations:
        num_nonan = np.min([num_iterations[0],np.sum(~np.isnan(dem))])
    else:
        # Calculate the total number of non-NaN cells
        num_nonan = np.sum(~np.isnan(dem))

    # Initialize lists to store upstream and downstream cell indices and flow contributions
    ix  = []
    ixc = []
    fc  = [] 
    
    # Iterate through all non-NaN cells
    for i in range(num_nonan-1):

        # Find the highest remaining cell in the temporary DEM
        high_cell_index = np.nanargmax(dem_temp)
        high_cell = np.unravel_index(high_cell_index, dem_temp.shape)
        
        # Calculate flow contributions from the highest cell to its neighbours
        flow_contributions = flow_contribution_neighbours(high_cell[0], high_cell[1], dem)
        
        # Define the neighbourhood directions (8-connected)
        neighbours = [(-1, -1), (-1, 0), (-1, 1),
                      (0, -1),           (0, 1),
                      (1, -1),  (1, 0),  (1, 1)]
        
        # Iterate over neighbours to update flow accumulation
        for j, (dx, dy) in enumerate(neighbours):
            nx = high_cell[0] + dx
            ny = high_cell[1] + dy
            
            if nx < dem.shape[0] and ny < dem.shape[1] and ~np.isnan(flow_contributions[j]) and flow_contributions[j] > 0:
                # Increment flow accumulation of the neighbour
                fa[nx, ny] += fa[high_cell] * flow_contributions[j]
                
                # Store indices of highest cell and neighbour, which define the flow direction
                ix.append(high_cell_index)
                ixc.append(high_cell_index + dx * dem.shape[1] + dy)
                fc.append(flow_contributions[j])

        # Mark the processed cell (highest cell) as inactive in the temporary DEM
        dem_temp[high_cell[0], high_cell[1]] = np.nan

    # Optional: Using a flow accumulation threshold, remove the pixels of the gully (uncomment if needed)
    #fa[fa > 1000] = np.nan

    return fa, ix, ixc, fc

### Load the filled DEM of the catchment of study
We use `rasterio` to manipulate raster datasets, extract information from them, and perform various raster operations.

In [4]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]
dem_files = []
for s in files:
    if 'asc' in s and 'dem' in s:
        dem_files.append(s)
input_file = widgets.Select(
    options=dem_files,
    description='dem files:',
    disabled=False
)
display(input_file)

Select(description='dem files:', options=('dem.asc', 'dem_fill.asc', 'dem_syn.asc', 'dem_syn_fill.asc', 'dem_s…

In [5]:
# Open the raster map file
with rasterio.open(input_file.value) as src:
    # Read the raster data as a numpy array
    dem_fill = src.read(1)  # Read the first band (index 0)

    # Get metadata of the raster map
    dem_metadata = src.meta

The `src.read(1)` function call reads the raster data as a numpy array. The `metadata` variable contains metadata information such as the raster's spatial reference system, data type, and geotransform. You can use this metadata for various purposes, such as georeferencing and understanding the properties of the raster map.

### Run the function to compute the flow accumulation using Dinf

In [57]:
fa_Dinf, ix_Dinf, ixc_Dinf, fc_Dinf = flow_accumulation_Dinf(dem_fill)

fig = px.imshow(fa_Dinf,color_continuous_scale='RdBu_r')
fig.show()

### Interactive plot

In [68]:
num_nonan = np.sum(~np.isnan(dem_fill))
def iplot_Dinf(num_iter=2):
    fa_Dinf, ix_Dinf, ixc_Dinf, fc_Dinf = flow_accumulation_Dinf(dem_fill,num_iter)

    fig = plt.subplots(figsize = (15,10))
    my_cmap = plt.cm.get_cmap('coolwarm')
    my_cmap.set_bad(alpha=0) # set how the colormap handles 'bad' values
    plt.imshow(fa_Dinf,cmap=my_cmap)
    plt.colorbar()
    plt.show()
    
# Create interactive widgets (sliders)
interact(iplot_Dinf, num_iter = IntSlider(2,2,num_nonan,layout=Layout(width='500px')))

interactive(children=(IntSlider(value=2, description='num_iter', layout=Layout(width='500px'), max=8822, min=2…

<function __main__.iplot_Dinf(num_iter=2)>

In [7]:
def check_fc(ix, fc):
    """
    Check if the sum of flow contributions from each upstream cell is equal to 1.

    Args:
        ix (list): List of upstream cell indices for each cell.
        fc (list): List of flow contributions for each cell.

    Raises:
        ValueError: If the sum of flow contributions from any upstream cell is not equal to 1.
    """
    # Iterate over unique upstream cell indices
    for source_index in np.unique(ix):
        # Extract flow contributions corresponding to the current upstream cell
        source_contributions = [fc[i] for i in range(len(ix)) if ix[i] == source_index]
        # Calculate the sum of flow contributions
        sum_contributions = sum(source_contributions)
        # Check if the sum is not close to 1 (within a small tolerance)
        if not np.isclose(sum_contributions, 1):
            # Raise ValueError if the sum is not equal to 1
            raise ValueError("Sum of flow contributions from the upstream cell {} is not equal to 1.".format(source_index))
            
    print("Sum of flow contributions from each upstream cell is equal to 1.")
    
check_fc(ix_Dinf, fc_Dinf)

Sum of flow contributions from each upstream cell is equal to 1.


### Save the flow accumulation as a raster file

In [15]:
if 'syn' in input_file.value:
    # Specify the output file path
    output_file = 'fa_Dinf_syn.asc'    
else:
    # Specify the output file path
    output_file = 'fa_Dinf.asc'

# Write the modified raster data to a new file with the same metadata
with rasterio.open(output_file, 'w', **dem_metadata) as dst:
    # Write the modified raster data to the new file
    dst.write(fa_Dinf, 1)  # Assuming raster_data is the modified array

C:\Users\Usuario\anaconda3\envs\iMPACt\lib\site-packages\rasterio\__init__.py:341: NotGeoreferencedWarning:

The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.

